<a href="https://colab.research.google.com/github/visiont3lab/flask-static-website/blob/master/Categorical_Varaibles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Esempio categorical variables

* [Reference Notebook Categorical Features](https://towardsdatascience.com/encoding-categorical-features-21a2651a065c)
* [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/chronic_kidney_disease)
* [Kaggle Kidney Deasease Dataset](https://www.kaggle.com/mansoordaku/ckdisease/data)
* [Chronic Kidney Deasease download](https://matthew-brett.github.io/cfd2019/data/chronic_kidney_disease)
* [Kidney Study Github](https://github.com/liuy14/Kidney_Disease_Detection)

In [ ]:
!wget https://matthew-brett.github.io/cfd2019/data/ckd_clean.csv

In [35]:
import pandas as pd
import numpy as np

In [75]:
# load data
df = pd.read_csv('ckd_clean.csv')
df.head(10)

Age  Blood Pressure  Specific Gravity  ...  Pedal Edema  Anemia Class
0  48.0            70.0             1.005  ...          yes     yes     1
1  53.0            90.0             1.020  ...           no     yes     1
2  63.0            70.0             1.010  ...          yes      no     1
3  68.0            80.0             1.010  ...          yes      no     1
4  61.0            80.0             1.015  ...          yes     yes     1
5  48.0            80.0             1.025  ...           no     yes     1
6  69.0            70.0             1.010  ...          yes      no     1
7  73.0            70.0             1.005  ...          yes      no     1
8  73.0            80.0             1.020  ...           no      no     1
9  46.0            60.0             1.010  ...           no      no     1

[10 rows x 25 columns]

## Extract Categorical Features

## Dummy Variables

In [88]:
# Get dummies
df_pandas_dummy = pd.get_dummies(df, prefix_sep='-', drop_first=False)
df_pandas_dummy.sort_index(axis=1, inplace=True)
# X head
df_pandas_dummy.head()

Age  Albumin  Anemia-no  ...  Specific Gravity  Sugar  White Blood Cell Count
0  48.0      4.0          0  ...             1.005    0.0                  6700.0
1  53.0      2.0          0  ...             1.020    0.0                 12100.0
2  63.0      3.0          1  ...             1.010    0.0                  4500.0
3  68.0      3.0          1  ...             1.010    2.0                 11000.0
4  61.0      2.0          0  ...             1.015    0.0                  9200.0

[5 rows x 35 columns]

Il parametro **prefix_sep='_'**  fa si che ogni classe abbia un nome unico separato da un separator. Il parametro **drop_first=True** cancella una delle due colonne allo scopo di evitare **multicollinearity**

## LabelEncoder + OneHotEncoding


In [91]:
# Categorical boolean mask
categorical_feature_mask = df.dtypes==object
#display(categorical_feature_mask)

# filter categorical columns using mask and turn it into a list
categorical_cols = df.columns[categorical_feature_mask].tolist()
categorical_cols

['Red Blood Cells',
 'Pus Cell',
 'Pus Cell clumps',
 'Bacteria',
 'Hypertension',
 'Diabetes Mellitus',
 'Coronary Artery Disease',
 'Appetite',
 'Pedal Edema',
 'Anemia']

### Label Encoder

> Importante: label encoder ordina i nomi in alfabeticamente

In [92]:
# Esempio
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])
list(le.classes_)
le.transform(["tokyo", "tokyo", "paris"])
list(le.inverse_transform([2, 2, 1]))

['tokyo', 'tokyo', 'paris']

In [93]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder

# apply le on categorical feature columns
cat_data = df[categorical_cols].apply(lambda col: le.fit_transform(col))
cat_data.head(10)

Red Blood Cells  Pus Cell  Pus Cell clumps  ...  Appetite  Pedal Edema  Anemia
0                1         0                1  ...         1            1       1
1                0         0                1  ...         1            0       1
2                0         0                1  ...         1            1       0
3                1         0                1  ...         1            1       0
4                0         0                0  ...         1            1       1
5                1         0                0  ...         0            0       1
6                1         0                0  ...         0            1       0
7                1         1                0  ...         0            1       0
8                0         0                0  ...         0            0       0
9                1         1                0  ...         0            0       0

[10 rows x 10 columns]

> Importante
Abbiamo degli output che sono 0 e 1 (binary) ma questo avviene perchè le nostre categorical features avevano solo due classi.

### One hot encoding

In [108]:
# import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
# instantiate OneHotEncoder
ohe = OneHotEncoder(sparse=False ) 

cat_data_ohe = ohe.fit_transform(cat_data.values) # It returns an numpy array

names = []
for name in categorical_cols:
    temp = df[name].unique()
    temp.sort()
    for n in temp:
        col_name = name+"-"+n #name.replace(" ","_")+"_"+n
        #print(col_name)
        names.append(col_name)

temp_df = df.drop(columns=categorical_cols)
temp_dff = pd.DataFrame(cat_data_ohe,dtype=np.float32,columns=names)
df_manual = pd.concat([temp_df, temp_dff], axis=1)
df_manual.sort_index(axis=1, inplace=True)
df_manual

Age  Albumin  Anemia-no  ...  Specific Gravity  Sugar  White Blood Cell Count
0    48.0      4.0        0.0  ...             1.005    0.0                  6700.0
1    53.0      2.0        0.0  ...             1.020    0.0                 12100.0
2    63.0      3.0        1.0  ...             1.010    0.0                  4500.0
3    68.0      3.0        1.0  ...             1.010    2.0                 11000.0
4    61.0      2.0        0.0  ...             1.015    0.0                  9200.0
..    ...      ...        ...  ...               ...    ...                     ...
153  55.0      0.0        1.0  ...             1.020    0.0                  6700.0
154  42.0      0.0        1.0  ...             1.025    0.0                  7800.0
155  12.0      0.0        1.0  ...             1.020    0.0                  6600.0
156  17.0      0.0        1.0  ...             1.025    0.0                  7200.0
157  58.0      0.0        1.0  ...             1.025    0.0                  6800.0

[158 rows x 35 columns]

## DictVectorized


In [89]:
#@title
# DictVectorizer
from sklearn.feature_extraction import DictVectorizer
# turn X into dict
X_dict = df.to_dict(orient='records') # turn each row as key-value pairs
# show X_dict
#X_dict[0:4]

# instantiate a Dictvectorizer object for X
dv_X = DictVectorizer(sparse=False, separator="-",sort=True) 
# sparse = False makes the output is not a sparse matrix

# apply dv_X on X_dict
X_encoded = dv_X.fit_transform(X_dict)
# show X_encoded
# vocabulary
#vocab = dv_X.vocabulary_

df_dictVect = pd.DataFrame(X_encoded,columns=dv_X.get_feature_names())
df_dictVect

Age  Albumin  Anemia-no  ...  Specific Gravity  Sugar  White Blood Cell Count
0    48.0      4.0        0.0  ...             1.005    0.0                  6700.0
1    53.0      2.0        0.0  ...             1.020    0.0                 12100.0
2    63.0      3.0        1.0  ...             1.010    0.0                  4500.0
3    68.0      3.0        1.0  ...             1.010    2.0                 11000.0
4    61.0      2.0        0.0  ...             1.015    0.0                  9200.0
..    ...      ...        ...  ...               ...    ...                     ...
153  55.0      0.0        1.0  ...             1.020    0.0                  6700.0
154  42.0      0.0        1.0  ...             1.025    0.0                  7800.0
155  12.0      0.0        1.0  ...             1.020    0.0                  6600.0
156  17.0      0.0        1.0  ...             1.025    0.0                  7200.0
157  58.0      0.0        1.0  ...             1.025    0.0                  6800.0

[158 rows x 35 columns]

## Check

In [114]:
print(sum(sum(df_manual.values - df_pandas_dummy.values)))
print(sum(sum(df_manual.values - df_dictVect.values)))


0.0
0.0


In [115]:
df_pandas_dummy.to_csv("dataset_ready.csv")